In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import os
from scipy import stats
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
df = pd.read_csv("/kaggle/input/sales-forecasting/train.csv",index_col = 0)

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
null_values = df.isnull().sum()
null_values

In [ ]:
null_rows = df[df.isnull().any(axis=1)]
null_rows

In [ ]:
df['Postal Code'] = df['Postal Code'].fillna(5401)
df.head()

In [ ]:
df['Order Date'] = pd.to_datetime(df['Order Date'], format='%d/%m/%Y')
df['Ship Date'] = pd.to_datetime(df['Ship Date'], format='%d/%m/%Y')
df.sort_values('Order Date', ascending=True, inplace=True)
df.head()

In [ ]:
exclude_columns = ["Customer Name", "City", "Order ID", "Customer ID", "Product ID","Product Name","Country"]
column_names = [col for col in df.select_dtypes(include=['object']).columns if col not in exclude_columns]

# Define the number of rows and columns for the subplots
num_rows = 3  
num_cols = (len(column_names) + num_rows - 1) // num_rows

# Create subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 15))

# Flatten the axes for easy iteration
axes = axes.flatten()

# Loop through each column and create a count plot
for i, col in enumerate(column_names):
    sns.countplot(x=df[col], ax=axes[i])
    axes[i].set_title(f"Count plot of {col}")
    axes[i].tick_params(axis='x', rotation=90)

# Remove any empty subplots
for i in range(len(column_names), num_rows * num_cols):
    fig.delaxes(axes[i])

# Adjust the layout
plt.tight_layout()
plt.show()

In [ ]:
df['Postal Code'] = df['Postal Code'].astype(int)
df.head()

In [ ]:
df['Order Month'] = df['Order Date'].dt.month
df['Order Year'] = df['Order Date'].dt.year
# Group and sum sales by year
yearly_sales = df.groupby('Order Year')['Sales'].sum().reset_index()

plt.figure(figsize=(12, 6))
plt.plot(yearly_sales['Order Year'], yearly_sales['Sales'], marker='o', linestyle='-', color='b')
plt.title('Yearly Sales Trends')
plt.xlabel('Year')
plt.ylabel('Sales')
plt.grid(True)
plt.xticks(rotation=45)
plt.show()

In [ ]:
df['Month-Year'] = df['Order Date'].dt.strftime('%B %Y')
monthly_sales = df.groupby('Month-Year')['Sales'].sum().reset_index()


# Plot monthly sales trends with year
plt.figure(figsize=(12, 6))
plt.plot(monthly_sales['Month-Year'], monthly_sales['Sales'], marker='o', linestyle='-', color='b')
plt.title('Monthly Sales Trends with Year')
plt.xlabel('Month-Year')
plt.ylabel('Sales')
plt.grid(True)
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Group and sum sales by year and region
yearly_region_sales = df.groupby(['Order Year', 'Region'])['Sales'].sum().reset_index()
best_performing_regions = yearly_region_sales.loc[yearly_region_sales.groupby('Order Year')['Sales'].idxmax()]
print("The best-performing regions by year are:")
print(best_performing_regions)



In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(df['Order Year'].astype(str) + ' ' + df['Region'], df['Sales'])
plt.xlabel('Year and Region')
plt.ylabel('Sales')
plt.title('Sales by Region for Each Year')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Extract quarter and year
df['Order Quarter'] = df['Order Date'].dt.quarter
df['Order Year'] = df['Order Date'].dt.year
# Group and sum sales by quarter and year
quarterly_sales = df.groupby(['Order Year', 'Order Quarter'])['Sales'].sum().reset_index()
# Create a new column for Quarter-Year
quarterly_sales['Quarter-Year'] = 'Q' + quarterly_sales['Order Quarter'].astype(str) + ' ' + quarterly_sales['Order Year'].astype(str)
# Plot quarterly sales trends with year
plt.figure(figsize=(12, 6))
plt.plot(quarterly_sales['Quarter-Year'], quarterly_sales['Sales'], marker='o', linestyle='-', color='b')
plt.title('Quarterly Sales Trends with Year')
plt.xlabel('Quarter-Year')
plt.ylabel('Sales')
plt.grid(True)
plt.xticks(rotation=45)
plt.show()

In [ ]:
average_sales_per_mode = df.groupby('Ship Mode')['Sales'].mean()
print("Average Sales per Order for Each Ship Mode:")
print(average_sales_per_mode)

In [ ]:
# Create a pie chart for average sales per order by ship mode
plt.figure(figsize=(5,5))
plt.pie(average_sales_per_mode, labels=average_sales_per_mode.index, autopct='%1.1f%%', startangle=140)
plt.title('Average Sales per Order for Each Ship Mode')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

In [ ]:
quarterly_segment_sales = df.groupby(['Order Quarter', 'Ship Mode'])['Sales'].sum().unstack(fill_value=0)

# Create a line plot for segment performance on a quarterly basis
plt.figure(figsize=(12, 6))
for segment in quarterly_segment_sales.columns:
    plt.plot(quarterly_segment_sales.index, quarterly_segment_sales[segment], marker='o', label=segment)

plt.xlabel('Quarter')
plt.ylabel('Total Sales')
plt.title('Ship Mode Performance on a Quarterly Basis')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
ship_count_by_city = df.groupby(['City', 'Ship Mode']).size().unstack(fill_value=0)

top_10_cities = ship_count_by_city.sum(axis=1).nlargest(10).index

# Filter the DataFrame to include only the top 10 cities
ship_count_top_10_cities = ship_count_by_city.loc[top_10_cities]

# Create a stacked bar plot to visualize segment preference in the top 10 cities
ship_count_top_10_cities.plot(kind='bar', stacked=True, figsize=(10, 6))
plt.xlabel('City')
plt.ylabel('Count')
plt.title('Ship Mode Preference in Top 10 Cities')
plt.legend(title='Ship Mode')
plt.xticks(rotation=45)
plt.show()

In [ ]:
pivot_table = pd.crosstab(df['Category'], df['Ship Mode'])

# Create a grouped bar plot
fig, ax = plt.subplots(figsize=(10, 6))

bar_width = 0.2
index = np.arange(len(pivot_table))

for i, segment in enumerate(pivot_table.columns):
    ax.bar(
        index + i * bar_width,
        pivot_table[segment],
        bar_width,
        label=segment
    )

ax.set_xlabel('Category')
ax.set_ylabel('Count')
ax.set_title('Ship Mode Counts Within Each Category')
ax.set_xticks(index + (bar_width * len(pivot_table.columns)) / 2)
ax.set_xticklabels(pivot_table.index)
ax.legend(title='Ship Mode')

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
yearly_segment_sales = df.groupby(['Order Year', 'Segment'])['Sales'].sum().reset_index()

# Find the segment with the highest sales for each year
best_segment_by_year = yearly_segment_sales.loc[yearly_segment_sales.groupby('Order Year')['Sales'].idxmax()]

print("Segment with Highest Sales in Each Year:")
print(best_segment_by_year)

In [ ]:
quarterly_segment_sales = df.groupby(['Order Quarter', 'Segment'])['Sales'].sum().unstack(fill_value=0)

# Create a line plot for segment performance on a quarterly basis
plt.figure(figsize=(12, 6))
for segment in quarterly_segment_sales.columns:
    plt.plot(quarterly_segment_sales.index, quarterly_segment_sales[segment], marker='o', label=segment)

plt.xlabel('Quarter')
plt.ylabel('Total Sales')
plt.title('Segment Performance on a Quarterly Basis')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
average_sales_by_segment = df.groupby('Segment')['Sales'].mean()

# Create a bar plot for average sales by segment
plt.figure(figsize=(10, 6))
average_sales_by_segment.plot(kind='bar', color='skyblue')
plt.xlabel('Segment')
plt.ylabel('Average Sales')
plt.title('Average Sales in Each Segment')
plt.xticks(rotation=0)
plt.show()

In [ ]:
segment_count_by_city = df.groupby(['City', 'Segment']).size().unstack(fill_value=0)

top_10_cities = segment_count_by_city.sum(axis=1).nlargest(10).index

# Filter the DataFrame to include only the top 10 cities
segment_count_top_10_cities = segment_count_by_city.loc[top_10_cities]

# Create a stacked bar plot to visualize segment preference in the top 10 cities
segment_count_top_10_cities.plot(kind='bar', stacked=True, figsize=(10, 6))
plt.xlabel('City')
plt.ylabel('Count')
plt.title('Segment Preference in Top 10 Cities')
plt.legend(title='Segment')
plt.xticks(rotation=45)
plt.show()


In [ ]:
segment_count_by_state = df.groupby(['State', 'Segment']).size().unstack(fill_value=0)

top_10_states = segment_count_by_state.sum(axis=1).nlargest(10).index


segment_count_top_10_states = segment_count_by_state.loc[top_10_states]

segment_count_top_10_states.plot(kind='bar', stacked=True, figsize=(10, 6))
plt.xlabel('City')
plt.ylabel('Count')
plt.title('Segment Preference in Top 10 States')
plt.legend(title='Segment')
plt.xticks(rotation=45)
plt.show()

In [ ]:
pivot_table = pd.crosstab(df['Category'], df['Segment'])

# Create a grouped bar plot
fig, ax = plt.subplots(figsize=(10, 6))

bar_width = 0.2
index = np.arange(len(pivot_table))

for i, segment in enumerate(pivot_table.columns):
    ax.bar(
        index + i * bar_width,
        pivot_table[segment],
        bar_width,
        label=segment
    )

ax.set_xlabel('Category')
ax.set_ylabel('Count')
ax.set_title('Segment Counts Within Each Category')
ax.set_xticks(index + (bar_width * len(pivot_table.columns)) / 2)
ax.set_xticklabels(pivot_table.index)
ax.legend(title='Segment')

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
category_info = df.groupby('Category').agg({
    'Sub-Category': pd.Series.unique,
    'Product Name': pd.Series.unique
}).reset_index()

# Print or display the subcategories and products within each category
for index, row in category_info.iterrows():
    print(f'Category: {row["Category"]}')
    print('Subcategories:', ', '.join(row["Sub-Category"]))
    print('Products:', ', '.join(row["Product Name"]))
    print()

In [ ]:
top_10_products_by_category = df.groupby('Category').apply(lambda x: x.nlargest(10, 'Sales')).reset_index(drop=True)

# Create a bar plot to visualize the top 10 products in each category
plt.figure(figsize=(12, 6))
for category, data in top_10_products_by_category.groupby('Category'):
    plt.bar(data['Product Name'], data['Sales'], label=category)

plt.xlabel('Product')
plt.ylabel('Sales')
plt.title('Top 10 Products Sold in Each Category')
plt.xticks(rotation=90)
plt.legend(title='Category')
plt.tight_layout()
plt.show()


In [ ]:
bottom_10_products_by_category = df.groupby('Category').apply(lambda x: x.nsmallest(10, 'Sales')).reset_index(drop=True)

# Create a bar plot to visualize the bottom 10 products in each category
plt.figure(figsize=(12, 6))
for category, data in bottom_10_products_by_category.groupby('Category'):
    plt.bar(data['Product Name'], data['Sales'], label=category)

plt.xlabel('Product')
plt.ylabel('Sales')
plt.title('Least Performing Products in Each Category')
plt.xticks(rotation=90)
plt.legend(title='Category')
plt.tight_layout()
plt.show()

In [ ]:
quarterly_category_sales = df.groupby(['Order Quarter', 'Category'])['Sales'].sum().unstack(fill_value=0)

# Create a line plot for category performance on a quarterly basis
quarters = df['Order Quarter'].unique()

plt.figure(figsize=(12, 6))
for category in quarterly_category_sales.columns:
    sales = quarterly_category_sales[category]
    plt.plot(quarters, sales, marker='o', label=category)

plt.xlabel('Quarter')
plt.ylabel('Total Sales')
plt.title('Category Performance on a Quarterly Basis')
plt.legend()
plt.grid(True)
plt.xticks(quarters)
plt.show()


In [ ]:
average_sales_by_category = df.groupby('Category')['Sales'].mean()

# Create a pie chart to visualize the average sales in each category
plt.figure(figsize=(5,5))
plt.pie(average_sales_by_category, labels=average_sales_by_category.index, autopct='%1.1f%%', startangle=140)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.title('Average Sales in Each Category')
plt.show()

In [ ]:
df['Time to Ship'] = (df['Ship Date'] - df['Order Date']).dt.days
df.head()

In [ ]:
correlation = np.corrcoef(df['Time to Ship'], df['Sales'])[0, 1]

# Print the correlation coefficient
print(f"Correlation Coefficient between Time to Ship and Sales: {correlation}")

A correlation coefficient close to 0 indicates a very weak or almost no linear relationship between the two variables. In this case, the negative sign indicates a very slight negative correlation, which means that as the time to ship increases slightly, sales might decrease slightly, but the relationship is very weak and likely not significant.

In [ ]:
f_statistic, p_value = stats.f_oneway(
    df[df['Ship Mode'] == 'Standard Class']['Time to Ship'],
    df[df['Ship Mode'] == 'Second Class']['Time to Ship'],
    df[df['Ship Mode'] == 'First Class']['Time to Ship'],
    df[df['Ship Mode'] == 'Same Day']['Time to Ship']
)

# Check the p-value to determine if there's a significant relationship
alpha = 0.05  # Set the significance level
if p_value < alpha:
    print("There is a significant relationship between Ship Mode and Time to Ship.")
else:
    print("There is no significant relationship between Ship Mode and Time to Ship.")

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Perform Tukey HSD test
tukey_result = pairwise_tukeyhsd(df['Time to Ship'], df['Ship Mode'])

# Print the results
print(tukey_result)

Here are the key findings:

There are significant differences in "Time to Ship" between all pairs of ship modes. All comparisons have p-adj values less than 0.05, indicating that the mean times to ship are significantly different between these ship modes.

Specifically, "Same Day" and "Standard Class" have the largest mean difference in "Time to Ship" ("meandiff" = 4.9638), and all comparisons involving these two ship modes are significant.

"First Class" and "Second Class" also have a significant difference in "Time to Ship" ("meandiff" = 1.07).

In summary, the Tukey HSD test reveals that there are significant differences in the mean times to ship between all pairs of ship modes. These differences indicate that the choice of ship mode is associated with variations in the time it takes to ship orders, and all of the differences are statistically significant.

In [ ]:
f_statistic, p_value = stats.f_oneway(
    df[df['Segment'] == 'Consumer']['Time to Ship'],
    df[df['Segment'] == 'Corporate']['Time to Ship'],
    df[df['Segment'] == 'Home Office']['Time to Ship']
)

# Check the p-value to determine if there's a significant relationship
alpha = 0.05  # Set the significance level
if p_value < alpha:
    print("There is a significant relationship between Segment and Time to Ship.")
else:
    print("There is no significant relationship between Segment and Time to Ship.")


In [ ]:
f_statistic, p_value = stats.f_oneway(
    df[df['Region'] == 'East']['Time to Ship'],
    df[df['Region'] == 'West']['Time to Ship'],
    df[df['Region'] == 'Central']['Time to Ship'],
    df[df['Region'] == 'South']['Time to Ship']
)

# Check the p-value to determine if there's a significant relationship
alpha = 0.05  # Set the significance level
if p_value < alpha:
    print("There is a significant relationship between Region and Time to Ship.")
else:
    print("There is no significant relationship between Region and Time to Ship.")

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Perform Tukey HSD test
tukey_result = pairwise_tukeyhsd(df['Time to Ship'], df['Region'])

# Print the results
print(tukey_result)

The results of the Tukey HSD test for multiple comparisons of means between different regions and their respective "Time to Ship" values are as follows:

There are three regions: Central, East, and West.
The test compares the means of "Time to Ship" between these regions.
The "meandiff" column represents the mean difference in "Time to Ship" between the compared regions.
The "p-adj" column represents the adjusted p-value for each comparison.
The "lower" and "upper" columns represent the lower and upper bounds of the confidence interval for the mean difference.
The "reject" column indicates whether to reject the null hypothesis (True) or not (False) for each comparison.
Here are the key findings:

There is a significant difference in "Time to Ship" between the Central and East regions, with the East region having a shorter time to ship ("meandiff" = -0.1556, p-adj < 0.05).

There is a significant difference in "Time to Ship" between the Central and West regions, with the West region having a shorter time to ship ("meandiff" = -0.1356, p-adj < 0.05).

There is no significant difference in "Time to Ship" between the East and South regions ("meandiff" = 0.051, p-adj > 0.05).

There is no significant difference in "Time to Ship" between the East and West regions ("meandiff" = 0.02, p-adj > 0.05).

There is no significant difference in "Time to Ship" between the South and West regions ("meandiff" = -0.0309, p-adj > 0.05).

In summary, the Tukey HSD test reveals that the Central and East regions have significantly shorter times to ship compared to the Central and West regions. However, there are no significant differences in time to ship between the East and South regions, East and West regions, and South and West regions. The "reject" column indicates whether the differences are statistically significant at the chosen significance level (FWER=0.05), with "True" indicating significance.

In [ ]:
max_time_to_ship_by_city_product = df.groupby(['City', 'Product Name'])['Time to Ship'].max().reset_index()

# Find the combination(s) with the maximum time to ship
max_time_to_ship = max_time_to_ship_by_city_product['Time to Ship'].max()

# Filter the DataFrame to include only the rows with the maximum time to ship
max_time_to_ship_data = max_time_to_ship_by_city_product[max_time_to_ship_by_city_product['Time to Ship'] == max_time_to_ship]

# Display the product names and cities where the time to ship is maximum
print(max_time_to_ship_data[['City', 'Product Name']])

In [ ]:
max_time_to_ship_data = df[df['Time to Ship'] == max_time_to_ship]

# Create a bar chart to visualize the results
plt.figure(figsize=(10, 6))
plt.bar(max_time_to_ship_data['State'], max_time_to_ship_data['Time to Ship'], color='skyblue')
plt.xlabel('State')
plt.ylabel('Time to Ship')
plt.title('States with Maximum Time to Ship by Product')
plt.xticks(rotation=90)  # Rotate x-axis labels for better readability
plt.tight_layout()
plt.show()

In [ ]:
average_time_to_ship_by_city = df.groupby('City')['Time to Ship'].mean().reset_index()

# Sort the cities in ascending order of average time to ship
sorted_cities = average_time_to_ship_by_city.sort_values(by='Time to Ship')

# Select the top 10 cities with the shortest shipping times
top_10_cities = sorted_cities.head(10)

# Display the top 10 cities with the shortest shipping times
print(top_10_cities)

In [ ]:

# Create a bar chart to visualize the results
plt.figure(figsize=(12, 6))
plt.barh(top_10_cities['City'], top_10_cities['Time to Ship'], color='skyblue')
plt.xlabel('Average Time to Ship')
plt.ylabel('City')
plt.title('Top 10 Cities with Shortest Average Time to Ship')
plt.gca().invert_yaxis()  # Invert the y-axis to display the shortest times at the top
plt.tight_layout()
plt.show()

In [ ]:
average_time_to_ship_by_state = df.groupby('State')['Time to Ship'].mean().reset_index()
plt.figure(figsize=(10, 6))
plt.bar(average_time_to_ship_by_state['State'], average_time_to_ship_by_state['Time to Ship'], color='skyblue')
plt.xlabel('State')
plt.ylabel('Average Time to Ship')
plt.title('Average Time to Ship by State')
plt.xticks(rotation=90)  # Rotate x-axis labels for better readability
plt.tight_layout()
plt.show()

In [ ]:
average_time_to_ship_by_category = df.groupby('Category')['Time to Ship'].mean().reset_index()
plt.figure(figsize=(10, 6))
plt.bar(average_time_to_ship_by_category['Category'], average_time_to_ship_by_category['Time to Ship'], color='skyblue')
plt.xlabel('Category')
plt.ylabel('Time to Ship (Days)')
plt.title('Average Time to Ship for Each Category')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
sales_by_subcategory_quarter = df.groupby(['Sub-Category', 'Order Quarter'])['Sales'].sum().reset_index()

# Pivot the data to have subcategories as columns and quarters as rows
pivot_sales = sales_by_subcategory_quarter.pivot(index='Order Quarter', columns='Sub-Category', values='Sales')

# Plot the results as a bar chart
pivot_sales.plot(kind='bar', figsize=(10, 6))
plt.xlabel('Quarter')
plt.ylabel('Sales')
plt.title('Sales of Subcategories in Each Quarter')
plt.xticks(rotation=45)
plt.legend(title='Subcategory')
plt.tight_layout()
plt.show()

In [ ]:

# 2. Customer Segmentation
# Segment customers by 'Segment'
segmented_customers = df['Segment'].value_counts()
plt.figure(figsize=(8, 6))
sns.barplot(x=segmented_customers.index, y=segmented_customers.values)
plt.title('Customer Segmentation')
plt.xlabel('Segment')
plt.ylabel('Number of Customers')
plt.show()

# 3. Customer Demographics Analysis
# Analyze customer demographics (e.g., region, state, city)
region_distribution = df['Region'].value_counts()
state_distribution = df['State'].value_counts()
city_distribution = df['City'].value_counts()

# 4. Customer Purchase Behavior Analysis
# Calculate metrics like average order value and total revenue
average_order_value = df.groupby('Customer ID')['Sales'].mean()
total_revenue = df.groupby('Customer ID')['Sales'].sum()
print(total_revenue)
# 5. Customer Segmentation Insights
# Develop customer profiles for each segment
segment_profiles = df.groupby('Segment')['Sales'].describe()
print(segment_profiles)
# 6. Customer Retention Analysis
# Calculate customer retention rates
retention_rates = df.groupby(['Customer ID', 'Order Date']).size().reset_index(name='Order Count')
retention_rates = retention_rates.groupby('Customer ID').count()
churn_rate = (1 - (retention_rates[retention_rates['Order Date'] > 1].shape[0] / df['Customer ID'].nunique())) * 100
print(churn_rate)
# 7. Cross-Selling and Upselling Opportunities
# Analyze which products are frequently purchased together
cross_sell_matrix = df.pivot_table(index='Order ID', columns='Product Name', values='Sales', aggfunc='sum', fill_value=0)
print(cross_sell_matrix)
# 8. Geographical Insights
# Determine geographic distribution of customers
plt.figure(figsize=(10, 5))
sns.countplot(data=df, x='Region')
plt.title('Customer Distribution by Region')
plt.xlabel('Region')
plt.ylabel('Number of Customers')
plt.show()

This is a relatively low churn rate, indicating good customer retention for the superstore. It's important to continue monitoring and engaging with customers to maintain or improve this retention rate.

In [ ]:
# Calculate total sales revenue
total_sales = df['Sales'].sum()

# Calculate total number of orders
total_orders = df['Order ID'].nunique()

# Calculate average purchase value
average_purchase_value = total_sales / total_orders
# Calculate total number of unique customers
total_customers = df['Customer ID'].nunique()

# Calculate average purchase frequency
average_purchase_frequency = total_orders / total_customers
customer_lifespan = 1 / churn_rate
CLV = average_purchase_value * average_purchase_frequency * customer_lifespan
CLV

This means that, on average, a customer is expected to generate around $1,739.64 in revenue over the one-year CLV period.

In [ ]:
N = 10  # You can change this value to show more or fewer products
product_performance = df.groupby('Product ID').agg({
    'Product Name': 'first',
    'Sales': 'sum',
}).reset_index()
best_selling_products = product_performance.sort_values(by='Sales', ascending=False)
least_selling_products = product_performance.sort_values(by='Sales')

# Select the top N best-selling products and their sales values
top_best_selling = best_selling_products.head(N)
top_best_selling_names = top_best_selling['Product Name']
top_best_selling_sales = top_best_selling['Sales']

# Select the top N least-selling products and their sales values
top_least_selling = least_selling_products.head(N)
top_least_selling_names = top_least_selling['Product Name']
top_least_selling_sales = top_least_selling['Sales']

# Create subplots for best-selling and least-selling products
plt.figure(figsize=(12, 6))

# Subplot for best-selling products
plt.subplot(1, 2, 1)
plt.barh(top_best_selling_names, top_best_selling_sales, color='skyblue')
plt.xlabel('Total Sales')
plt.title(f'Top {N} Best-Selling Products')
plt.gca().invert_yaxis()  # Invert the y-axis to show the top products at the top

# Subplot for least-selling products
plt.subplot(1, 2, 2)
plt.barh(top_least_selling_names, top_least_selling_sales, color='salmon')
plt.xlabel('Total Sales')
plt.title(f'Top {N} Least-Selling Products')
plt.gca().invert_yaxis()  # Invert the y-axis to show the top products at the top

# Adjust spacing between subplots
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import warnings

# Assuming you have a DataFrame 'df' with 'Customer ID,' 'Order Frequency,' and other relevant features
# Assuming you have a DataFrame called 'df' with 'Customer ID' and 'Order Date' columns
# Convert 'Order Date' to a datetime format if it's not already in one
df['Order Date'] = pd.to_datetime(df['Order Date'])

# Group the data by 'Customer ID' and count the number of unique orders
order_frequency = df.groupby('Customer ID')['Order Date'].nunique().reset_index()

# Rename the column to 'Order Frequency'
order_frequency = order_frequency.rename(columns={'Order Date': 'Order Frequency'})

# Merge the calculated order frequency back into the main DataFrame
df = df.merge(order_frequency, on='Customer ID', how='left', suffixes=('', '_freq'))


# Select the features you want to use for clustering (e.g., 'Order Frequency' and 'Sales')
X = df[['Order Frequency', 'Sales']]

# Choose the number of clusters (you can experiment with different values)
num_clusters = 3

# Initialize the K-Means model with n_init set explicitly to suppress the warning
kmeans = KMeans(n_clusters=num_clusters, random_state=0, n_init=10)

# Fit the model to your data
kmeans.fit(X)

# Add the cluster labels back to your DataFrame
df['Cluster'] = kmeans.labels_

# Visualize the clusters
plt.figure(figsize=(10, 6))
scatter = plt.scatter(df['Order Frequency'], df['Sales'], c=df['Cluster'], cmap='rainbow')
plt.xlabel('Order Frequency')
plt.ylabel('Sales')
plt.title('Customer Segmentation')
# Get unique cluster labels
unique_labels = df['Cluster'].unique()

# Create a legend with the cluster labels
legend_labels = ['Cluster ' + str(label) for label in unique_labels]
plt.legend(handles=scatter.legend_elements()[0], labels=legend_labels)

plt.show()


In [ ]:
# Calculate cluster statistics
cluster_stats = df.groupby('Cluster')[['Order Frequency', 'Sales']].mean().reset_index()

# Add cluster size
cluster_size = df['Cluster'].value_counts().reset_index()
cluster_size.columns = ['Cluster', 'Count']
cluster_stats = cluster_stats.merge(cluster_size, on='Cluster')

# Define cluster labels
cluster_labels = {
    0: 'High Frequency, High Sales',
    1: 'Low Frequency, Low Sales',
    2: 'Medium Frequency, Medium Sales'
}

# Add cluster labels
cluster_stats['Cluster Label'] = cluster_stats['Cluster'].map(cluster_labels)

# Create a tabloid or table
cluster_stats_tabloid = cluster_stats[['Cluster', 'Cluster Label', 'Order Frequency', 'Sales', 'Count']]

# Display the tabloid
cluster_stats_tabloid


Based on the K-Means clustering analysis of customer data, we have identified three distinct customer segments:

Cluster 0 - High Frequency, High Sales:

Customers in this segment have a high order frequency (approximately 7.21 orders) and high sales (approximately $134.54).
This cluster is the largest, containing 9,305 customers.
Cluster 1 - Low Frequency, Low Sales:

Customers in this segment have a low order frequency (approximately 6.33 orders) and low sales (approximately $10,608.89).
This cluster is the smallest, with only 15 customers.
Cluster 2 - Medium Frequency, Medium Sales:

Customers in this segment exhibit medium order frequency (approximately 7.19 orders) and medium sales (approximately $1,771.95).
This cluster includes 480 customers.
These findings provide valuable insights into the different customer behaviors within the dataset. For instance:

Cluster 0 represents a large group of customers who frequently make purchases with relatively high sales. These customers are likely loyal and contribute significantly to revenue.

Cluster 1 includes a small group of customers who make infrequent and low-value purchases. It may be essential to engage with this segment to increase their activity or explore reasons for their limited engagement.

Cluster 2 comprises customers with moderate purchase frequency and spending. Understanding their needs and preferences can help tailor marketing strategies.

Overall, these customer segments allow for more targeted marketing, product recommendations, and customer retention efforts, ultimately contributing to business growth and optimization.

In [ ]:
df1 = pd.read_csv("/kaggle/input/sales-forecasting/train.csv",index_col = 0)
df1['Postal Code'] = df1['Postal Code'].fillna(5401).astype('int')
df1.drop(['Order ID','Customer ID','Customer Name','Product ID'],axis = 1, inplace = True)

In [ ]:
df1.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

# Assuming 'df1' is your DataFrame with label-encoded categorical columns
non_numerical_columns = df1.select_dtypes(exclude=['number']).columns

# Initialize a LabelEncoder
label_encoder = LabelEncoder()

# Encode non-numerical columns
for column in non_numerical_columns:
    df1[column + '_Encoded'] = label_encoder.fit_transform(df1[column])
    df1.drop(column, axis=1, inplace=True)
df1.head()

In [ ]:
from xgboost import XGBRegressor
# Define your features and target variable
X = df1.drop(columns=['Sales'])  # Features
y = df1['Sales']  # Target variable

# Split the data into training and testing sets (e.g., 80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)
xgb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_model.predict(X_test)

# Calculate Mean Squared Error (MSE) as the evaluation metric
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)